In [35]:
import joblib
import numpy as np
import bm25s
import os
from sentence_transformers import SentenceTransformer
import pandas as pd
from dateutil import parser

In [36]:
def format_date(date_string):
  date_object = parser.parse(date_string)
  formatted_date = date_object.strftime("%Y-%m-%d")
  return format_date


def read_dataframe(path):
  df = pd.read_csv(path)

  df['published_at'] = df["published_at"].apply(format_date)
  df['updated_at'] = df['updated_at'].apply(format_date)


  df = df.to_dict(orient='records')
  return df


In [37]:
news_data = read_dataframe('news_data_dedup.csv')

In [38]:
news_data[0]

{'guid': 'e3dc5caa18f9a16d7edcc09f8d5c2bb4',
 'title': "Harvey Weinstein's 2020 rape conviction overturned",
 'description': 'Victims group describes the New York appeal court\'s decision to retry Hollywood mogul as "profoundly unjust".',
 'venue': 'BBC',
 'url': 'https://www.bbc.co.uk/news/world-us-canada-68899382',
 'published_at': <function __main__.format_date(date_string)>,
 'updated_at': <function __main__.format_date(date_string)>}

In [ ]:
def query_news(indecis):
  output = [news_data[x] for x in indecis]
  return output

In [34]:
query_news([1,2,3])

[{'guid': '297b7152cd95e80dd200a8e1997e10d9',
  'title': 'Police and activists clash on Atlanta campus amid Gaza protests',
  'description': 'Meanwhile, hundreds of students march in Washington DC, and congresswoman Ilhan Omar joins protesters at a New York campus.',
  'venue': 'BBC',
  'url': 'https://www.bbc.co.uk/news/live/world-us-canada-68898923',
  'published_at': <function __main__.format_date(date_string)>,
  'updated_at': <function __main__.format_date(date_string)>},
 {'guid': '170bd18d1635c44b9339bdbaf1e62123',
  'title': 'Haiti PM resigns as transitional council sworn in',
  'description': 'The council will try to restore order and form a new government in the nation gripped by gang violence.',
  'venue': 'BBC',
  'url': 'https://www.bbc.co.uk/news/world-latin-america-68896323',
  'published_at': <function __main__.format_date(date_string)>,
  'updated_at': <function __main__.format_date(date_string)>},
 {'guid': 'e696224ac208878a5cec8bdc9f97c632',
  'title': 'Europe risks 

In [45]:
corpus = [x['title'] + " " + x['description'] for x in news_data]
BM25_retriever = bm25s.BM25(corpus=corpus)
tokenize_data = bm25s.tokenize(corpus)
BM25_retriever.index(tokenize_data)

sample_query = "What are the recent news about GDP?"
tokenize_sample_query = bm25s.tokenize(sample_query)

result, score = BM25_retriever.retrieve(tokenize_sample_query, k=3)

print("Sample query: " + sample_query)
for doc in result[0]:
  print(f"Document retrieved: {corpus.index(doc)}: {doc}/n" )

Split strings:   0%|          | 0/870 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/870 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Sample query: What are the recent news about GDP?
Document retrieved: 752: GDP and the Dow Are Up. But What About American Well-Being? The standard ways of measuring economic growth don’t capture what life is like for real people. A new metric offers a better alternative, especially for seeing disparities across the country./n
Document retrieved: 673: What the GDP Report Says About Inflation: A Hot First Quarter Thursday’s gross domestic product report suggests that a widely watched inflation reading due Friday could be worse than expected./n
Document retrieved: 289: A GDP Warning as Signs of Stagflation Appear Slower growth and persistent inflation explain why voters feel glum about the economy./n


In [46]:
corpus = [x['title'] + " " + x['description'] for x in news_data]
BM25_RETRIEVER = bm25s.BM25(corpus=corpus)
TOKENIZED_DATA = bm25s.tokenize(corpus)
BM25_RETRIEVER.index(TOKENIZED_DATA)

Split strings:   0%|          | 0/870 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/870 [00:00<?, ?it/s]

In [68]:
def bm25_retrieve(query: str, top_k: int = 5):

    ### START CODE HERE ###

    # Tokenize the query using the 'tokenize' function from the 'bm25s' module
    tokenized_query = bm25s.tokenize(query)

    # Index the tokenized chunks with the retriever
    BM25_RETRIEVER.index(TOKENIZED_DATA)

    # Use the 'BM25_RETRIEVER' to retrieve documents and their scores based on the tokenized query
    # Retrieve the top 'k' documents
    results, scores = BM25_RETRIEVER.retrieve(tokenized_query, k=top_k)

    # Extract the first element from 'results' to get the list of retrieved documents
    results = results[0]

    # Convert the retrieved documents into their corresponding indices in the results list
    top_k_indices = [corpus.index(doc) for doc in results]

    ### END CODE HERE ###

    return top_k_indices


In [69]:
bm25_retrieve(sample_query, 3)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

[752, 673, 289]

In [70]:
EMBEDDINGS = joblib.load('embeddings.joblib')

In [71]:
model = SentenceTransformer("BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [73]:
query = "RAG is awesome"
model.encode(query)[:40]

array([ 0.0088631 , -0.04775134, -0.00156082,  0.01309995, -0.00206928,
       -0.0615726 ,  0.01384691,  0.00101497, -0.04903949, -0.04762556,
       -0.03628183,  0.00478039, -0.03492185,  0.05323149,  0.02193956,
        0.03645146,  0.04029361, -0.00453634,  0.01883799, -0.0336738 ,
        0.02516185, -0.04843625, -0.04047941,  0.02590899,  0.02175236,
        0.03160365,  0.03937941, -0.03640447, -0.03113303, -0.01247232,
        0.03661651, -0.00458202, -0.00100167, -0.03188784,  0.02957142,
        0.01986151, -0.00737466,  0.02370171, -0.02151624, -0.07361353],
      dtype=float32)

In [87]:
def cosine_similarity(query_embeddings, array_of_vectors):
  v1 = np.array(query_embeddings)
  similarities = []
  if len(np.shape(array_of_vectors)) == 1:
    array_of_vectors = [array_of_vectors]

  for v2 in array_of_vectors:
    v2 = np.array(v2)
    similarities.append(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
  return np.array(similarities)

In [88]:
query1 = "What are the primary colors"
query2 = "Yellow, red and blue"
query3 = "Cats are friendly animals"

query1_embed = model.encode(query1)
query2_embed = model.encode(query2)
query3_embed = model.encode(query3)

In [90]:
print(f"similarity between {query1} and {query2}: {cosine_similarity(query1_embed, query2_embed)[0]}")
print(f"similarity between {query1} and {query3}: {cosine_similarity(query1_embed, query3_embed)[0]}")


similarity between What are the primary colors and Yellow, red and blue: 0.7377147674560547
similarity between What are the primary colors and Cats are friendly animals: 0.45086297392845154


In [91]:
query = "Taylor Swift"
query_embed = model.encode(query)

similarity_score = cosine_similarity(query_embed, EMBEDDINGS)

In [93]:
similarity_indices = np.argsort(-similarity_score)

In [97]:
top_2 = similarity_indices[:2]

In [98]:
top_2

array([350, 176])

In [99]:
query_news(top_2)

[{'guid': '927257674585bb6ef669cf2c2f409fa7',
  'title': '‘The working class can’t afford it’: the shocking truth about the money bands make on tour',
  'description': 'As Taylor Swift tops $1bn in tour revenue, musicians playing smaller venues are facing pitiful fees and frequent losses. Should the state step in to save our live music scene?When you see a band playing to thousands of fans in a sun-drenched festival field, signing a record deal with a major label or playing endlessly from the airwaves, it’s easy to conjure an image of success that comes with some serious cash to boot – particularly when Taylor Swift has broken $1bn in revenue for her current Eras tour. But looks can be deceiving. “I don’t blame the public for seeing a band playing to 2,000 people and thinking they’re minted,” says artist manager Dan Potts. “But the reality is quite different.”Post-Covid there has been significant focus on grassroots music venues as they struggle to stay open. There’s been less focus on

In [107]:
def semantic_search_retrieve(query, top_k=5):
  query_embed = model.encode(query)
  similarity_score = cosine_similarity(query_embed, EMBEDDINGS)

  similarity_indices = np.argsort(-similarity_score)
  top_k_indices = similarity_indices[:top_k]
  top_k_indices = [int(top) for top in top_k_indices]

  return top_k_indices

In [108]:
semantic_search_retrieve("What are the recent news about GDP?")

[743, 673, 626, 752, 326]

In [110]:
def reciprocal_rank_fusion(list1, list2, top_k , K=60):
  rrf_scores = {}

  for lst in [list1, list2]:
    for rank, item in enumerate(lst, start=1):
      if item not in rrf_scores:
        rrf_scores[item] = 0

      rrf_scores[item] += 1 / (rank + K)

    sorted_items = sorted(rrf_scores.keys(), key=rrf_scores.get, reverse = True)

    top_k_indices = [int(x) for x in sorted_items[:top_k]]

    return top_k_indices



In [132]:
def prompt_genate(query, retrieval_type ,top_k=5):
  if retrieval_type == "reciprocal_rank_fusion":
    list1 = bm25_retrieve(query, top_k)
    list2 = semantic_search_retrieve(query, top_k)
    hybrid = reciprocal_rank_fusion(list1, list2, top_k)
    top_k_indices = hybrid

  elif retrieval_type == "bm25":
    top_k_indices = bm25_retrieve(query, top_k)
  elif retrieval_type == "semantic_search":
    top_k_indices = semantic_search_retrieve(query, top_k)

  documents = query_news(top_k_indices)
  formatted_documents = []
  for document in documents:
    formatted_document = (
            f"Title: {document['title']}, Description: {document['description']}, "
            f"Published at: {document['published_at']}\nURL: {document['url']}"
  )
    formatted_documents.append(formatted_document)
  retrieve_data_format = "\n".join(formatted_documents)

  prompt = (
        f"Answer the user query below. There will be provided additional information for you to compose your answer. "
        f"The relevant information provided is from 2024 and it should be added as your overall knowledge to answer the query, "
        f"you should not rely only on this information to answer the query, but add it to your overall knowledge."
        f"Query: {query}\n"
        f"2024 News: {retrieve_data_format}"
  )

  return prompt

In [134]:
from google.colab import userdata
key = userdata.get('GROQ_API')

llm = ChatGroq(model="deepseek-r1-distill-llama-70b",
               api_key=key,
               temperature=0,
               max_tokens=None,
               reasoning_format="parsed",
               timeout=None,
               max_retries=2)





In [135]:
def llm_response(query, retrieval_type, top_k=5):
  prompt = prompt_genate(query, retrieval_type, top_k)
  res = llm.invoke(prompt)
  return res.content


In [138]:
responsebm25 = llm_response("What are the recent news about GDP?", "bm25", 5)
responsesementic = llm_response("What are the recent news about GDP?", "semantic_search", 5)
responsehybrid = llm_response("What are the recent news about GDP?", "reciprocal_rank_fusion", 5)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/870 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

In [139]:
print(responsebm25)

The recent news about GDP from 2024 highlights several key points:

1. **GDP Growth and Stock Market Performance**: The GDP and the Dow Jones Industrial Average (Dow) have shown growth, indicating a positive economic trajectory. However, this growth does not necessarily reflect the well-being of all Americans, as traditional economic metrics like GDP may not capture the full picture of living conditions across different regions and demographics.

2. **Inflation Concerns**: A recent GDP report suggests that inflation may be higher than expected in the first quarter of 2024. This could impact future economic policies and consumer confidence.

3. **Signs of Stagflation**: There are concerns about slower economic growth coupled with persistent inflation, which could lead to a period of stagflation. This combination of slower growth and higher prices might explain why many voters feel pessimistic about the economy.

4. **Limitations of GDP as a Metric**: There is a growing discussion about 

In [140]:
print(responsesementic)

The recent news about GDP indicates that the U.S. economy experienced a slowdown in the first quarter of 2024, with the gross domestic product (GDP) expanding at a 1.6% seasonally- and inflation-adjusted annual rate. This growth rate reflects a deceleration compared to previous quarters, signaling a potential cooling of economic activity.

Additionally, the GDP report has raised concerns about upcoming inflation readings. Analysts suggest that the data could indicate higher-than-expected inflation in the near term, which may impact future economic policies and market dynamics.

There is also a discussion about whether GDP and other traditional economic indicators, such as the Dow Jones Industrial Average, accurately reflect the well-being of Americans. Some argue that these metrics do not fully capture the real-life experiences and disparities across the country, prompting calls for alternative measures that better represent the broader population's economic reality.


In [141]:
print(responsehybrid)

The recent news about GDP from 2024 highlights several key points:

1. **GDP Growth and Stock Market Performance**: The GDP and the Dow Jones Industrial Average (Dow) have shown growth, indicating a positive economic trajectory. However, this growth does not necessarily reflect the well-being of all Americans, as traditional economic metrics like GDP may not capture the full picture of living conditions across different regions and demographics.

2. **Inflation Concerns**: A recent GDP report suggests that inflation may be higher than expected in the first quarter of 2024. This could impact future economic policies and consumer confidence.

3. **Signs of Stagflation**: There are concerns about slower economic growth coupled with persistent inflation, which could lead to a period of stagflation. This combination of slower growth and higher prices might explain why many voters feel pessimistic about the economy.

4. **Limitations of GDP as a Metric**: There is a growing discussion about 

In [124]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="deepseek-r1-distill-llama-70b",
               api_key=key,
               temperature=0,
               max_tokens=None,
               reasoning_format="parsed",
               timeout=None,
               max_retries=2)

In [128]:
res=llm.invoke(prompt)

In [130]:
print(res.content)

Here are some recent news items in technology and related fields, along with their sources:

1. **ByteDance Considers Shutting Down TikTok in the US**
   - ByteDance, the parent company of TikTok, may shut down the app in the US rather than sell it, citing the core importance of its algorithms to overall operations.
   - Source: [The Guardian](https://www.theguardian.com/technology/2024/apr/25/bytedance-shut-down-tiktok-than-sell)

2. **Biden Administration Streamlines Power-Grid Approvals**
   - The Biden administration is consolidating authority over large power-grid projects to expedite upgrades and access to new clean-energy projects.
   - Source: [The Wall Street Journal](https://www.wsj.com/politics/policy/energy-department-gets-power-over-big-grid-projects-3865d42d)

3. **Australia Provides $100 Million Aid to Ukraine**
   - Australia has announced a $100 million aid package for Ukraine, including funding and supplies, as part of international support efforts.
   - Source: [News